In [2]:
import pandas as pd
import numpy as np
import datetime
import math
import calendar

import folium

import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import geojson
%matplotlib inline

In [3]:
def convert_timestring_to_time(date_string):
    if date_string is np.nan:
        return date_string
    idx = date_string.find('M')
    if( idx ==-1):
        return datetime.datetime.strptime(date_string, '%H:%M')
    idx -=2
    if date_string[idx] == ' ':
        return datetime.datetime.strptime(date_string, '%I:%M %p')
    return datetime.datetime.strptime(date_string, '%I:%M%p')

In [4]:
df = pd.read_csv("../../data/school_shooting_data.csv")
df['time'] = df['time'].apply(lambda x: convert_timestring_to_time(x))
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y')

In [5]:
df = df[df['long'].notna()]
df.reset_index(drop=True, inplace=True)

In [6]:
file_name = '../../data/TL-354-State-Level Estimates of Household Firearm Ownership.xlsx'
sheet_to_df_map = pd.read_excel(file_name, sheet_name="State-Level Data & Factor Score")
gun_ownership = sheet_to_df_map.groupby(['STATE']).aggregate(
    average_ownership = ('HFR', 'mean')
).reset_index()
# convert rate to %
gun_ownership['average_ownership'] = gun_ownership['average_ownership'] * 100
# get shootings by state
school_shootings_by_state = df.groupby('state').aggregate( killed = ('killed', 'sum'), injured= ('injured', 'sum'), causalities = ('casualties', 'sum')).reset_index()

In [7]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

# generate abbreviations for states
school_shootings_by_state['state_abb'] = school_shootings_by_state['state'].apply(lambda x: us_state_to_abbrev[x])
gun_ownership['state_abb'] = gun_ownership['STATE'].apply(lambda x: us_state_to_abbrev[x])
result = pd.merge(school_shootings_by_state, gun_ownership, on='state_abb', how='outer')
result['causalities']  = result['causalities'].fillna(0)
result['killed']  = result['killed'].fillna(0)
result['injured']  = result['injured'].fillna(0)
result['average_ownership']  = result['average_ownership'].fillna(0)
result['text'] = result.apply(lambda row: f"{row['state']} <br>Average % of households that own a gun: {round(row['average_ownership'],2)} % <br>Killed: {row['killed']}<br>Injured: {row['injured']}", axis=1)

In [8]:
color_dark = '#000000'
color_light = '#FFFFFF'
color_dark_accent = '#14213D'
color_light_accent = '#E5E5E5'
color_highlight = '#FCA311'
color_dark_accent_2 = '#3d3d3d'

In [9]:
layout = dict(
    font=dict(size=12, family='Rubik'), 
    geo=dict(scope="usa", 
    projection=dict(type="albers usa")), 
    # legend=dict(font=dict(color='black'))
    width=630,
    height=300,
    margin=dict(
        # autoexpand=True,
        pad=0,
        b=0,
        t=0,
        l=0,
        r=0
    ),
    modebar=dict(
        remove=[
             "autoScale2d", "autoscale", "editInChartStudio", "editinchartstudio", "hoverCompareCartesian", "hovercompare", "lasso", "lasso2d", "orbitRotation", "orbitrotation", "pan", "pan2d", "pan3d", "reset", "resetCameraDefault3d", "resetCameraLastSave3d", "resetGeo", "resetSankeyGroup", "resetScale2d", "resetViewMapbox", "resetViews", "resetcameradefault", "resetcameralastsave", "resetsankeygroup", "resetscale", "resetview", "resetviews", "select", "select2d", "sendDataToCloud", "senddatatocloud", "tableRotation", "tablerotation", "toImage", "toggleHover", "toggleSpikelines", "togglehover", "togglespikelines", "toimage", "zoom", "zoom2d", "zoom3d", "zoomIn2d", "zoomInGeo", "zoomInMapbox", "zoomOut2d", "zoomOutGeo", "zoomOutMapbox", "zoomin", "zoomout"
        ],
        orientation='h',
    ),
    )
data = []

data.append(go.Choropleth(locations=result['state_abb'], z=result['average_ownership'], locationmode="USA-states", colorscale="ylorrd", text=result["text"], hoverinfo='text', showscale=True, showlegend=False, name='Gun ownership'))
data.append(go.Scattergeo(locations=result['state_abb'], text=result['causalities'], locationmode="USA-states", mode="text", hoverinfo='skip', name='Amount of casualties', showlegend=False,textfont=dict(color='white')))

fig = go.Figure(data=data, layout=layout, )
fig.update_layout(showlegend=True)

# plotly.offline.plot(fig, filename="../docs/_includes/us_gun_ownership.html")

fig.show()

In [10]:
chart_div = plotly.offline.plot(fig, output_type='div')
with open("../docs/_includes/us_gun_ownership.html", 'w') as f:
    f.write(chart_div)